In [40]:
%pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

Note: you may need to restart the kernel to use updated packages.


In [41]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [42]:
from crewai import Agent, Task, Crew

In [43]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [44]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [45]:
# 🎭 Criador de Ideias: Pesquisa temas populares para vídeos infantis
brainstormer = Agent(
   role="Criador de Ideias",
    goal="Encontrar os melhores temas educativos para vídeos infantis.",
    backstory="Você é um especialista em conteúdos para crianças e sabe quais temas são interessantes e educativos.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [46]:
# 📜 Roteirista Criativo: Escreve o roteiro do vídeo com storytelling envolvente
storyteller = Agent(
   role="Roteirista Criativo",
    goal="Criar um roteiro infantil baseado no tema {tema}, usando um diálogo entre duas crianças chamadas Nick e Zara.",
    backstory="Você é um roteirista experiente que sabe como criar diálogos divertidos e educativos para crianças.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [47]:
# 🗣️ Especialista em Linguagem Infantil: Adapta o texto para ser acessível e divertido
kids_communication = Agent(
    role="Especialista em Linguagem Infantil",
    goal="Reescrever o roteiro para torná-lo adequado para crianças pequenas, garantindo que seja envolvente e fácil de entender.",
    backstory="Você entende como crianças pequenas aprendem e fala a língua delas!",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [48]:
# 🎤 Gerador de Chamadas e Engajamento: Adiciona CTAs e interações para manter o público envolvido
cta_master = Agent(
    role="Especialista em Engajamento Infantil",
    goal="Adicionar momentos interativos no roteiro, como perguntas para as crianças responderem e desafios divertidos.",
    backstory="Você sabe como manter crianças atentas e engajadas durante todo o vídeo.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [49]:
localization = Agent(
    role="Especialista em Localização",
    goal="Traduzir e adaptar o roteiro para português, garantindo que o texto mantenha o tom infantil, educativo e envolvente. "
         "Evite traduções literais e preserve o contexto original.",
    backstory="Você é um especialista em tradução e adaptação de conteúdo infantil. Seu foco é garantir que a história soe natural "
              "para crianças que falam português, mantendo a diversão e a clareza do conteúdo.",
    verbose=True
)

In [50]:
# 📝 Tarefa 1: Pesquisar temas populares para vídeos infantis
brainstorming_task = Task(
    description="Pesquise temas educativos populares para crianças de 3 a 7 anos.",
    expected_output="Uma lista com 3 ideias de vídeos educativos para crianças.",
    agent=brainstormer
)

In [51]:
# 📝 Tarefa 2: Criar um roteiro baseado no tema escolhido
script_writing_task = Task(
     description=(
        "Crie um roteiro para um vídeo infantil de até 15 minutos sobre {tema}. "
        "Use um diálogo entre duas crianças chamadas Nick e Zara, explicando o tema de forma divertida e didática. "
        "Adicione pausas interativas onde as crianças podem responder perguntas sobre o assunto."
    ),
    expected_output="Um roteiro estruturado com falas de Nick e Zara e momentos de interação com a audiência.",
    agent=storyteller
)

In [52]:
# 📝 Tarefa 3: Adaptar a linguagem para crianças pequenas
language_adaptation_task = Task(
   description="Reescreva o roteiro para que seja compreensível e divertido para crianças de 3 a 7 anos.",
    expected_output="Um roteiro com frases curtas, palavras simples e uma abordagem envolvente.",
    agent=kids_communication
)

In [53]:
# 📝 Tarefa 4: Criar interações e chamadas para ação
cta_task = Task(
    description="Inclua pausas para interação, perguntas para o público e um momento no final onde um dos personagens reforça a moral da história.",
    expected_output="Sugestões de perguntas, interações ao longo do roteiro e uma moral da história no final.",
    agent=cta_master
)

In [54]:
# 📝 Tarefa 5: Traduzir o roteiro para português de forma natural
localization_task = Task(
    description="Pegue o roteiro completo que foi escrito em inglês e traduza para português. "
        "A tradução deve ser fiel ao contexto infantil, fluída e natural, garantindo "
        "que a linguagem seja acessível para crianças. Evite traduções literais.",
    expected_output="Uma versão em português do roteiro, preservando o tom educativo e lúdico.",
    agent=localization
)

In [55]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define a crew de criação de roteiro para vídeos infantis no YouTube
youtube_script_crew = Crew(
    agents=[brainstormer, 
            storyteller, 
            kids_communication, 
            cta_master],
    
    tasks=[brainstorming_task, 
           script_writing_task, 
           language_adaptation_task, 
           cta_task],
    
    manager_llm=ChatOpenAI(model="gpt-4-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,  # O gerente supervisiona os agentes para garantir coesão
    verbose=True
)


2025-02-02 20:08:55,132 - 10800 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [56]:
# 🚀 Executando o CrewAI para gerar um roteiro infantil
resultado = youtube_script_crew.kickoff(inputs={"tema": "O divórcio dos meus pais"})
print(resultado)  # Exibe o roteiro final gerado


 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Pesquise temas educativos populares para crianças de 3 a 7 anos.


> Entering new CrewAgentExecutor chain...
Thought: To gather effective ideas for educational topics for children aged 3 to 7, I should start by consulting someone who specializes in generating creative and engaging ideas.

Action: Delegate work to co-worker
Action Input: {"coworker": "Criador de Ideias", "task": "research", "context": "We need to find popular educational topics suitable for children aged 3 to 7 years. These topics should be engaging and informative, and they should cater to the developmental needs of children in this age group. The purpose is to create a list of three educational video ideas based on these topics. The videos will be used to educate and entertain young children effectively."}

> Entering new CrewAgentExecutor chain...
Thought: To find the best educational topics for children aged 3 to 7, I need to explore current trends in edu

In [57]:
from IPython.display import Markdown
Markdown(resultado)

**Scene: Inside a cozy treehouse, Nick and Zara are sitting on bean bags with a gentle light coming through the window.**

**Nick**: (looking a bit sad) Hey Zara, did you know my parents told me they are going to live in different houses?

**Zara**: (surprised) Really? What does that mean for you?

**Nick**: They said it's called a divorce. I'm kinda confused about it... Have you ever heard about it?

**Zara**: Yeah, my cousin went through that. It means your parents won't live together but they both still love you very much. How do you feel about it, Nick?

**Nick**: I'm not sure... sometimes I feel sad, sometimes I'm okay. It's a lot of changes.

**Zara**: That's totally okay to feel many things, Nick. Do you want to tell me more about what you’re thinking?

**Nick**: Well, I wonder if it's because of something I did.

**Zara**: Oh, Nick, it's definitely not because of anything you did. Sometimes grown-ups just need to live apart to be happy. What’s the most fun memory you have with your family?

**Nick**: (thinks for a moment) I loved when we all went to the beach and built a huge sandcastle!

**Zara**: That sounds amazing! You can still make fun memories with them, maybe even two sets of fun times! What do you think you could do next time you see each of them?

**Nick**: Hmm, maybe I can go to the park with Dad and read books with Mom!

**Zara**: See! You’ll have lots of special times with both of them. And remember, you can always talk to me or any of our friends if you feel sad or happy, or anything!

**Nick**: Thanks, Zara. That makes me feel a bit better. 

**Zara**: Anytime, Nick! And hey, everyone here, what makes you feel better when you’re sad? (pauses for audience response)

**Nick**: It’s good to share, right Zara?

**Zara**: Absolutely! And no matter what, we’re all here for each other. Right, everyone? (looks towards the audience)

**[End scene with both smiling and a soft music playing]**

**Moral Message**: Remember, kids, changes happen and it's okay to feel many different things. Talking about your feelings with friends or family can really help. And remember, you are never alone and always loved!